# 代码生成
Instruct模型在代码生成方面也非常有效，得到特定条件的约束后，模型就可以根据我们的要求生成对应的代码， 我们所熟知的Copilot 就是一个很好的示例。通过巧妙的提示，你可以执行大量的代码生成任务。下面我们来看几个示例。

我们可以先实现一个简单的用户欢迎程度，用Python代码实现，在提示词中加以约束：

In [1]:
### 加载模型Qwen模型
from modelscope import AutoModelForCausalLM, AutoTokenizer

model_name_or_path_instruct = '/home/lixinyu/weights/Qwen2.5-3B-Instruct'  # 替换为你下载的模型路径
tokenizer_instruct = AutoTokenizer.from_pretrained(model_name_or_path_instruct)
model_instruct = AutoModelForCausalLM.from_pretrained(model_name_or_path_instruct,device_map='cuda:0', torch_dtype='auto')

/home/lixinyu/miniconda3/envs/prompt/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/home/lixinyu/miniconda3/envs/prompt/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.18it/s]


In [2]:
# 推理函数
def inference(model,tokenizer,prompt,system="你是一个专业的人工智能助手",max_new_tokens=512,temperature=0.9):
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        do_sample=True,
    )
    # 提取仅由模型生成的token ids（排除输入部分）
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    # 统计生成的tokens数量（直接取生成序列的长度）
    generated_tokens_count = len(generated_ids[0])  # 因输入是单条，取第一个元素的长度
    # 解码生成的token ids为文本
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    # 返回响应文本和生成的tokens数量
    return {
        "response":response,
        "generated_tokens_count":generated_tokens_count
    }

In [3]:
prompt_1="""
按照下面的要求只生成python代码:
/*
打印出“hello world”
*/
"""

In [4]:
print(inference(model_instruct,tokenizer_instruct,prompt_1))

{'response': '```python\nprint("hello world")\n```', 'generated_tokens_count': 10}


使用Qwen模型生成的结果中包含有(```python …… ```)，事实上，这是markdown语言中代码块的表述，出现在此的原因是Qwen对于生成代码会生成结构化回答，而这个结构化刚好是markdown格式，其实并不影响实际使用，我们只需要复制粘贴其中的代码部分即可。事实上，其他的代码语言均是如此，我们看下一个例子，实现一个 SQL 查询的描述，用于从两个表中检索属于 "计算机科学系" 的所有学生信息。

In [7]:
prompt_2="""
只使用MySQL查询来回答这个问题：
Table departments, columns = [DepartmentId, DepartmentName]
Table students, columns = [DepartmentId, StudentId, StudentName]
为计算机科学系的所有学生创建一个MySQL查询的代码
"""

In [8]:
print(inference(model_instruct,tokenizer_instruct,prompt_2))

{'response': '要为计算机科学系的所有学生创建一个MySQL查询，首先需要确定“计算机科学系”的 `DepartmentId`。假设计算机科学系的 `DepartmentId` 是 `1`，那么可以使用以下SQL查询来获取该系所有学生的记录：\n\n```sql\nSELECT StudentId, StudentName \nFROM students \nWHERE DepartmentId = 1;\n```\n\n如果 `DepartmentId` 不是 `1`，请将 `1` 替换为实际的 `DepartmentId` 值。', 'generated_tokens_count': 106}


上面的例子都是提供了些基础信息，那比如我们简单写一个问题，不包含任何格式、代码要求，希望模型写出代码，比如：

In [11]:
prompt_3="""怎样用 Python代码读取一个 CSV 文件并画出销售额随时间变化的折线图？仅请写出代码，不要有其他内容"""

In [12]:
print(inference(model_instruct,tokenizer_instruct,prompt_3))


{'response': "```python\nimport pandas as pd\nimport matplotlib.pyplot as plt\n\n# 读取CSV文件\ndf = pd.read_csv('sales_data.csv')\n\n# 假设CSV文件中包含两列：日期和销售额\n# 确保日期列是索引，并转换为日期格式\ndf.set_index('Date', inplace=True)\ndf.index = pd.to_datetime(df.index)\n\n# 获取销售额数据\nsales = df['Sales']\n\n# 绘制销售额随时间变化的折线图\nplt.figure(figsize=(10, 5))\nplt.plot(sales, marker='o')\nplt.title('Sales Over Time')\nplt.xlabel('Date')\nplt.ylabel('Sales')\nplt.grid(True)\nplt.show()\n```", 'generated_tokens_count': 150}
